In [9]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [10]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

e0301bb8-bca4-42dc-9f4d-d82da55bce0f
<class 'pinecone.models.serverless_spec.ServerlessSpec'>


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '449d4b2791e826c1556343eec393be80', 'Date': 'Thu, 22 Aug 2024 18:24:10 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [11]:
import json
data = json.load(open('reviews.json'))
data["reviews"]

[{'professor': 'Dr. Emily Foster',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Foster is an amazing professor! She explains complex topics clearly and is always willing to help outside of class. Highly recommend her courses.'},
 {'professor': 'Dr. James Mitchell',
  'subject': 'Mathematics',
  'stars': 3,
  'review': "Dr. Mitchell knows his stuff, but his lectures can be dry. You need to put in a lot of effort to keep up, but he's fair in grading."},
 {'professor': 'Prof. Susan Reynolds',
  'subject': 'History',
  'stars': 4,
  'review': 'Prof. Reynolds is very knowledgeable and engaging. Her lectures are well-organized, but she gives a lot of reading assignments.'},
 {'professor': 'Dr. Brian Lee',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Dr. Lee is very difficult to follow in lectures. He goes through material too fast, and the exams are extremely tough.'},
 {'professor': 'Dr. Linda Patel',
  'subject': 'Physics',
  'stars': 5,
  'review': "Dr. Patel is

In [12]:
processed_data = []
client = OpenAI()

for review in data["reviews"]:
    response = client.embeddings.create(
        input=review["review"],
        model="text-embedding-3-small",

    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "subject": review["subject"],
            "review": review["review"],
            "stars": review["stars"]
        }
    })

In [13]:
processed_data[0]

{'values': [-0.020623937249183655,
  -0.019648808985948563,
  0.01180513110011816,
  0.04563594609498978,
  0.020587369799613953,
  0.016772184520959854,
  -0.008538455702364445,
  0.015931136906147003,
  -0.00440940260887146,
  -0.011689335107803345,
  0.029083164408802986,
  -0.027571717277169228,
  0.005616122391074896,
  -0.025987135246396065,
  -0.019441595301032066,
  0.03988269343972206,
  -0.014407501555979252,
  0.01604083925485611,
  0.031813520938158035,
  0.05611855909228325,
  0.01683313027024269,
  -0.02925381064414978,
  0.00473850779235363,
  -0.014200286939740181,
  -0.05948274955153465,
  -0.06343201547861099,
  0.024000313133001328,
  0.02515827678143978,
  -0.00241648661904037,
  0.010214455425739288,
  0.07488975673913956,
  -0.009568434208631516,
  -0.01437093410640955,
  -0.023866234347224236,
  -0.05743498355150223,
  0.022001303732395172,
  0.0011800560168921947,
  0.05992155522108078,
  0.013249538838863373,
  0.008928506635129452,
  0.02111149951815605,
  -0.

In [14]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [15]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}